<a href="https://colab.research.google.com/github/hun9008/ML_TeamProject_24SS/blob/main/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/ML_TeamProject/

/content/drive/MyDrive/ML_TeamProject


In [3]:
ls

data/             images_png/                                        VGG.ipynb
images_gray_jpg/  preprocessed_images_40/                            무_train_labeled_2000/
images_gray_png/  PreTest.ipynb
images_jpg/       vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
!pip install -q keras

In [7]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 12.4 MB/s eta 0:00:00


In [19]:
!pip install tensorflow

In [6]:
import os
os.chdir("/content/drive/My Drive/ML_TeamProject/images_jpg")
!ls

incipient_train_labeled_2000  no_train_labeled_2000
mature_train_labeled_2000     overripe_train_labeled_2000


In [15]:
ls

incipient_train_labeled_2000/  no_train_labeled_2000/        TEST/   VAL/
mature_train_labeled_2000/     overripe_train_labeled_2000/  TRAIN/


In [ ]:
##
# 생략 가능(폴더 이미 만들었다면)
##
import os
import random
import shutil

# 데이터 폴더 경로 설정
data_dir = '/content/drive/My Drive/ML_TeamProject/images_jpg/'  # 원래의 데이터 폴더 경로
train_dir = '/content/drive/My Drive/ML_TeamProject/images_jpg/TRAIN'
val_dir = '/content/drive/My Drive/ML_TeamProject/images_jpg/VAL'
test_dir = '/content/drive/My Drive/ML_TeamProject/images_jpg/TEST'

# 각 클래스 이름 설정
class_names = ['overripe', 'no', 'mature', 'incipient']

# 각 클래스 폴더에 대해 작업 수행
for class_name in class_names:
    class_path = os.path.join(data_dir, class_name)
    print(class_path)
    if os.path.isdir(class_path):
        # 클래스 폴더 내 이미지 파일 리스트 가져오기
        images = os.listdir(class_path)

        # 이미지 파일을 랜덤하게 섞기
        random.shuffle(images)

        # 데이터 분할 인덱스 계산
        num_images = len(images)
        print(num_images)
        num_train = int(0.7 * num_images)
        num_val = int(0.15 * num_images)
        num_test = num_images - num_train - num_val

        # 데이터 분할하여 해당 폴더에 복사
        for i, image in enumerate(images):
            src = os.path.join(class_path, image)
            if i < num_train:
                dst = os.path.join(train_dir, class_name)
            elif i < num_train + num_val:
                dst = os.path.join(val_dir, class_name)
            else:
                dst = os.path.join(test_dir, class_name)
            # 실제 이미지 파일을 복사합니다.
            shutil.copy(src, dst)


/content/drive/My Drive/ML_TeamProject/images_jpg/overripe
2001


In [20]:
import numpy as np
from PIL import Image

# 데이터 폴더 경로 설정
train_dir = 'TRAIN'
val_dir = 'VAL'
test_dir = 'TEST'

# 각 데이터 폴더에서 이미지를 로드하고 numpy 배열로 변환
def load_images_from_folder(folder):
    images = []
    labels = []
    for class_folder in os.listdir(folder):
        class_path = os.path.join(folder, class_folder)
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = Image.open(img_path)
            # 이미지 크기 조정 및 색상 조정
            img = img.resize((224, 224))  # 예시로 224x224 크기로 resize
            img = np.array(img)  # 이미지를 numpy 배열로 변환
            images.append(img)
            # 레이블 지정
            labels.append(class_folder)
    return np.array(images), np.array(labels)

# TRAIN, VAL, TEST 데이터를 각각 로드
x_train, y_train = load_images_from_folder(train_dir)
x_val, y_val = load_images_from_folder(val_dir)
x_test, y_test = load_images_from_folder(test_dir)

# 각 데이터의 shape 확인
print("Train data shape:", x_train.shape)
print("Train labels shape:", y_train.shape)
print("Validation data shape:", x_val.shape)
print("Validation labels shape:", y_val.shape)
print("Test data shape:", x_test.shape)
print("Test labels shape:", y_test.shape)


NotADirectoryError: [Errno 20] Not a directory: 'TRAIN/crop_D14_0305bbba-60a5-11ec-8402-0a7404972c70.jpg'

In [ ]:
import optuna
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

def objective(trial):
    # 하이퍼파라미터 범위 설정
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 10, 50)

    # VGG16 모델 불러오기
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    # Fully-connected 레이어 추가
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # 모델 구성
    model = Model(inputs=base_model.input, outputs=predictions)

    # 모델 컴파일
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # 모델 학습
    model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_images, val_labels), verbose=0)

    # 검증 데이터에 대한 정확도 계산
    accuracy = model.evaluate(val_images, val_labels, verbose=0)[1]

    return accuracy